# Data Management

## Setup

In [1]:
# Import modules
import ast
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import Counter

from scipy.spatial.distance import cdist

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances

from tqdm import tqdm


In [2]:
# Load data
df = pd.read_csv('/Users/julienmbarki/Documents/Doctorat/Publications/Article 2/Data/Code/data_management/db_soundcharts/major_tracks_info_21-22.csv')


## Clean data

In [3]:
# Collapse to weekly level playlists data
# Determine weekday
df['collection_date'] = pd.to_datetime(df['collection_date'])

df['weekday'] = df['collection_date'].dt.weekday

# Filter to keep only rows that fall on Fridays
df_weekly = df[df['weekday'] == 4].copy()
df_weekly.drop(columns=['weekday'], inplace=True)

# Print data
display(df_weekly)


,playlist_id,playlist_name,collection_date,playlist_type,track_position,track_name,track_uuid,first_appearance_date,artist_name,artist_id,artist_first_track,artist_isnew,label_name,label_type,track_date,track_features,isrc,track_genres
0,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,1,Trucs de choses,a162dcf0-87c2-40c9-bd29-c46b1b3a17c4,2021-10-01 00:00:00+00:00,"['Gradur', 'Franglish']","['11e81bbb-3230-33fe-9244-a0369fe50396', '11e8...","['2010-01-01T00:00:00+00:00', '2016-01-01T00:0...","[False, False]",['Lutèce Music'],['unknown'],2021-07-29T00:00:00+00:00,"{'acousticness': 0.32, 'danceability': 0.54, '...",FR,['hip-hop & rap']
1,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,2,Ça canarde,f508505b-2950-4060-a1d8-ebbcfee16c86,2021-10-01 00:00:00+00:00,"['Guy2Bezbar', 'Bolémvn']","['11e81bbe-10f1-7a10-a0dd-a0369fe50396', '11e8...","['2016-06-03T00:00:00+00:00', '1995-03-21T00:0...","[False, False]",['Binks'],['Self released'],2021-07-08T00:00:00+00:00,"{'acousticness': 0.17, 'danceability': 0.76, '...",FR,['hip-hop & rap']
2,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,3,L'immortale,275eade0-a887-46e9-b027-5c96e4693dd6,2021-10-01 00:00:00+00:00,['Lacrim'],['11e81bd3-5427-df94-be96-a0369fe50396'],['2010-01-01T00:00:00+00:00'],[False],['Plata o Plomo'],"['Other independent labels: ""Indie""']",2021-09-22T00:00:00+00:00,"{'acousticness': 0.07, 'danceability': 0.89, '...",FR,['hip-hop & rap']
3,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,4,De bon matin feat. Guy2bezbar,71213346-3f56-4e11-b413-14409ba6877b,2021-10-01 00:00:00+00:00,"['Guy2Bezbar', 'Niska']","['11e81bbe-10f1-7a10-a0dd-a0369fe50396', '11e8...","['2016-06-03T00:00:00+00:00', '2013-11-12T00:0...","[False, False]",['Talent Factory'],['Self released'],2021-08-19T00:00:00+00:00,"{'acousticness': 0.17, 'danceability': 0.77, '...",FR,"['hip-hop & rap', 'pop']"
4,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,5,Daddy chocolat,41c7d316-2787-404e-9030-aabdca99fc28,2021-10-01 00:00:00+00:00,"['Koba LaD', 'Gazo']","['11e8ab5d-3039-fbac-9221-525400009efb', '59a6...","['1900-01-01T00:00:00+00:00', '1900-01-01T00:0...","[False, False]","['Millenium', 'Grinta']","['Universal', 'Other independent labels: ""Indi...",2021-06-17T00:00:00+00:00,"{'acousticness': 0.07, 'danceability': 0.72, '...",FR,['hip-hop & rap']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2579102,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,106,Acapulco,bf677c95-c9eb-4e09-a815-b42c80a705f6,2021-10-01 00:00:00+00:00,['Jason Derulo'],['11e81bcc-e6ef-4616-a1c3-a0369fe50396'],['2005-03-09T00:00:00+00:00'],[False],['Derulo'],['Self released'],2021-09-03T00:00:00+00:00,"{'acousticness': 0.05, 'danceability': 0.77, '...",US,['pop']
2579103,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,107,Mon Amour - Remix,29a0db26-4828-45dd-9299-4ea8d659c107,2021-12-04 00:00:00+00:00,"['Aitana', 'zzoilo']","['11e83fea-f894-b5e4-9144-a0369fe50396', '8702...","['2014-05-26T00:00:00+00:00', '2020-12-30T00:0...","[False, True]",['Voltereta'],['Self released'],2021-08-18T00:00:00+00:00,"{'acousticness': 0.12, 'danceability': 0.75, '...",ES,"['latin', 'pop']"
2579104,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,108,Sans moi,be9cc621-a5fb-4096-82cd-912367f155c5,2021-10-01 00:00:00+00:00,"['Franglish', 'Aya Nakamura']","['11e81bbb-8b2f-cc8a-98a2-a0369fe50396', '11e8...","['2016-01

In [4]:
# Normalize track_features
# Convert track features to dict
def string_to_list(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return []

df_weekly['track_features'] = df_weekly['track_features'].apply(string_to_list)
df_weekly['track_features'] = df_weekly['track_features'].apply(dict)

# Normalize features
df_features = pd.json_normalize(df_weekly['track_features'])
print(df_features)

# Concatenate the new columns back to the original DataFrame
df_final = pd.concat(
    [df_weekly.reset_index(drop=True), df_features.reset_index(drop=True)],
    axis=1
)
df_final.drop(columns='track_features', inplace=True)

# Print and export data
df_final.to_csv('major_tracks_info_21-22_weekly_normalized.csv', index=False)
display(df_final)


        acousticness  danceability  energy  instrumentalness   key  liveness  \
0               0.32          0.54    0.58              0.00   9.0      0.27   
1               0.17          0.76    0.74              0.00   6.0      0.27   
2               0.07          0.89    0.66              0.03   7.0      0.12   
3               0.17          0.77    0.80              0.00   0.0      0.30   
4               0.07          0.72    0.52              0.00   7.0      0.28   
...              ...           ...     ...               ...   ...       ...   
374662          0.05          0.77    0.79              0.00  10.0      0.16   
374663          0.12          0.75    0.76              0.00   2.0      0.10   
374664          0.15          0.71    0.65              0.00   1.0      0.09   
374665          0.00          0.73    0.69              0.00   0.0      0.10   
374666          0.01          0.57    0.69              0.00   6.0      0.17   

        loudness  mode  speechiness   t

,playlist_id,playlist_name,collection_date,playlist_type,track_position,track_name,track_uuid,first_appearance_date,artist_name,artist_id,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,timeSignature,valence
0,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,1,Trucs de choses,a162dcf0-87c2-40c9-bd29-c46b1b3a17c4,2021-10-01 00:00:00+00:00,"['Gradur', 'Franglish']","['11e81bbb-3230-33fe-9244-a0369fe50396', '11e8...",...,0.58,0.00,9.0,0.27,-4.73,0.0,0.19,96.66,5.0,0.63
1,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,2,Ça canarde,f508505b-2950-4060-a1d8-ebbcfee16c86,2021-10-01 00:00:00+00:00,"['Guy2Bezbar', 'Bolémvn']","['11e81bbe-10f1-7a10-a0dd-a0369fe50396', '11e8...",...,0.74,0.00,6.0,0.27,-5.98,0.0,0.28,150.12,4.0,0.61
2,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,3,L'immortale,275eade0-a887-46e9-b027-5c96e4693dd6,2021-10-01 00:00:00+00:00,['Lacrim'],['11e81bd3-5427-df94-be96-a0369fe50396'],...,0.66,0.03,7.0,0.12,-6.13,1.0,0.14,115.02,4.0,0.39
3,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,4,De bon matin feat. Guy2bezbar,71213346-3f56-4e11-b413-14409ba6877b,2021-10-01 00:00:00+00:00,"['Guy2Bezbar', 'Niska']","['11e81bbe-10f1-7a10-a0dd-a0369fe50396', '11e8...",...,0.80,0.00,0.0,0.30,-6.53,1.0,0.27,138.00,4.0,0.71
4,11e84480-ad44-deb2-8ac8-a0369fe50396,"Rap Fr | Rap Francais 2024 | Hits Rap | SDM, ...",2021-10-01 00:00:00+00:00,Major label,5,Daddy chocolat,41c7d316-2787-404e-9030-aabdca99fc28,2021-10-01 00:00:00+00:00,"['Koba LaD', 'Gazo']","['11e8ab5d-3039-fbac-9221-525400009efb', '59a6...",...,0.52,0.00,7.0,0.28,-8.40,1.0,0.15,141.03,4.0,0.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374662,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,106,Acapulco,bf677c95-c9eb-4e09-a815-b42c80a705f6,2021-10-01 00:00:00+00:00,['Jason Derulo'],['11e81bcc-e6ef-4616-a1c3-a0369fe50396'],...,0.79,0.00,10.0,0.16,-4.02,1.0,0.05,122.06,4.0,0.51
374663,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,107,Mon Amour - Remix,29a0db26-4828-45dd-9299-4ea8d659c107,2021-12-04 00:00:00+00:00,"['Aitana', 'zzoilo']","['11e83fea-f894-b5e4-9144-a0369fe50396', '8702...",...,0.76,0.00,2.0,0.10,-6.62,1.0,0.10,116.04,4.0,0.36
374664,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,108,Sans moi,be9cc621-a5fb-4096-82cd-912367f155c5,2021-10-01 00:00:00+00:00,"['Franglish', 'Aya Nakamura']","['11e81bbb-8b2f-cc8a-98a2-a0369fe50396', '11e8...",...,0.65,0.00,1.0,0.09,-4.45,1.0,0.33,91.79,4.0,0.42
374665,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,109,Better Days,c1f601c2-785f-41cd-8171-f8d421fd27a8,2022-01-30 00:00:00+00:00,"['J Balvin', 'NEIKED', 'Mae Muller', 'Polo G']","['11e81bca-6acb-9a46-8802-a0369fe50396', '11e8...",...,0.69,0.00,0.0,0.10,-5.09,0.0,0.04,110.03,4.0,0.75


## PCA dimensionnality reduction

In [5]:
# Load data
df = pd.read_csv('major_tracks_info_21-22_weekly_normalized.csv')

# Drop NAs
selected_columns = [
    "danceability", "energy", "loudness", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", 
    "tempo"
]
df = df.dropna(subset=selected_columns)
print(len(df))

# Group by playlist_name
groups = df.groupby(['playlist_name', 'collection_date'], as_index=False)


333394


In [6]:
# Select the number of components
scaler = MinMaxScaler()

selected_columns = [
    "danceability", "energy", "loudness", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", 
    "tempo"
]

optimal_num_components = []

for group_name, group in groups:
    # Subset and scale
    subset = group.loc[:, selected_columns]
    scaled_columns = scaler.fit_transform(subset)

    # Apply PCA
    pca = PCA()
    pca.fit(scaled_columns)
    
    # Calculate cumulative explained variance
    cumulative_variance = pca.explained_variance_ratio_.cumsum()
    optimal_components = next(i for i, var in enumerate(cumulative_variance) if var >= 0.8) + 1
    optimal_num_components.append(optimal_components)
    print(group_name, optimal_components)

    # Plot scree plot
    '''
    plt.plot(range(1, pca.n_components_ + 1), cumulative_variance, 'bo-', linewidth=2)
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance Ratio')
    plt.title(group_name)
    plt.show()
    '''

# Calculate the most modal value
most_common_optimal = Counter(optimal_num_components).most_common(1)
most_modal_value = most_common_optimal[0][0]

print("Most modal value of optimal components:", most_modal_value)


('#NUITVIE', '2021-10-01 00:00:00+00:00') 5
('#NUITVIE', '2021-10-08 00:00:00+00:00') 5
('#NUITVIE', '2021-10-15 00:00:00+00:00') 5
('#NUITVIE', '2021-10-22 00:00:00+00:00') 5
('#NUITVIE', '2021-10-29 00:00:00+00:00') 5
('#NUITVIE', '2021-11-05 00:00:00+00:00') 5
('#NUITVIE', '2021-11-12 00:00:00+00:00') 5
('#NUITVIE', '2021-11-19 00:00:00+00:00') 5
('#NUITVIE', '2021-11-26 00:00:00+00:00') 5
('#NUITVIE', '2021-12-03 00:00:00+00:00') 5
('#NUITVIE', '2021-12-10 00:00:00+00:00') 5
('#NUITVIE', '2021-12-17 00:00:00+00:00') 5
('#NUITVIE', '2021-12-24 00:00:00+00:00') 5
('#NUITVIE', '2021-12-31 00:00:00+00:00') 5
('#NUITVIE', '2022-01-07 00:00:00+00:00') 5
('#NUITVIE', '2022-01-14 00:00:00+00:00') 5
('#NUITVIE', '2022-01-21 00:00:00+00:00') 5
('#NUITVIE', '2022-01-28 00:00:00+00:00') 5
('#NUITVIE', '2022-02-04 00:00:00+00:00') 5
('#NUITVIE', '2022-02-11 00:00:00+00:00') 5
('#NUITVIE', '2022-02-18 00:00:00+00:00') 5
('#NUITVIE', '2022-02-25 00:00:00+00:00') 5
('#NUITVIE', '2022-03-04 00:00:0

In [7]:
# Apply PCA to reduce dimensionality
# Loop over each group and apply PCA with optimal number of components
scaler = MinMaxScaler()

selected_columns = [
    "danceability", "energy", "loudness", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", 
    "tempo"
]

reduced_data_dict = {}

for group_name, group in groups:
    # Subset data
    subset = group.loc[:, selected_columns]

    # Scale the specified columns
    scaled_columns = scaler.fit_transform(subset)

    # Apply PCA with the optimal number of components
    n_components = 5
    pca = PCA(n_components=n_components)
    reduced_data = pca.fit_transform(scaled_columns)
    reduced_data_dict[group_name] = reduced_data


## Clustering

### Elbow method

In [ ]:
# Apply the elbow method to determine the optimal number of clusters
# Calculate the elbow
wcss = {}

for k in range(1, 11):
  for group_name, group in groups:
    reduced_data = reduced_data_dict[group_name]
    
    kmeans = KMeans(n_clusters=k, random_state=42).fit(reduced_data)
    
    if group_name not in wcss:
      wcss[group_name] = []

    wcss[group_name].append(kmeans.inertia_)

# Plot the elbow
for group_name, values in wcss.items():
  plt.plot(range(1, 11), values, label=group_name)
  plt.xlabel('Number of clusters (k)')
  plt.ylabel('Within-cluster sum of squares (WCSS)')
  plt.legend()
  plt.show()


### Gap statistic method

In [8]:
# Set random seeds
random.seed(42)
np.random.seed(42)


In [9]:
def compute_gap(data, k):
    # Compute the WCSS for the real data
    kmeans = KMeans(n_clusters=k, random_state=42).fit(data)
    wcss = kmeans.inertia_

    # Compute the null reference distribution by shuffling the data and
    # re-assigning it to clusters
    n_samples, n_features = data.shape
    wcss_null = []

    for _ in range(20):
        data_shuffled = np.random.permutation(data)
        wcss_null.append(KMeans(n_clusters=k).fit(data_shuffled).inertia_)
  
    wcss_null = np.array(wcss_null)
  
    # Compute the gap statistic and gap*
    gap = np.mean(np.log(wcss_null)) - np.log(wcss)

    # Compute the standard deviation of the null reference distribution
    gap_std = np.std(np.log(wcss_null))

    return gap, gap_std


In [10]:
# Calculate the optimal k
# Loop over each group and generate scree plot
optimal_num_k = {}
optimal_num_k_2 = {}

for group_name, group in tqdm(groups):
    group_gaps = []
    group_errors = []

    for k in range(1, 11):
        reduced_data = reduced_data_dict[group_name]

        # Compute the gap statistic and standard deviation for the current value of k
        gap, gap_std = compute_gap(reduced_data, k)
        group_gaps.append(gap)
        group_errors.append(gap_std)

    # Find the optimal number of components based on the gap statistic criterion
    optimal_k = None
    for i in range(1, len(group_gaps) - 1):
        s_k = group_errors[i]
        threshold = s_k * np.sqrt(1 + 1 / 20)
        if group_gaps[i] >= group_gaps[i + 1] - threshold:
            optimal_k = i + 1
            break

    if optimal_k is None:
        optimal_k = np.argmax(group_gaps) + 1

    optimal_num_k[group_name] = optimal_k

    # Find the optimal number of components based on the gap* statistic criterion
    for i in range(1, len(group_gaps)):
        optimal_k_2 = np.argmax(group_gaps) + 1

    optimal_num_k_2[group_name] = optimal_k_2

    # Plot the gap statistics
    '''
    plt.plot(range(1, 11), group_gaps, label=group_name)

    # Set labels and display the plot
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Gap statistic')
    plt.legend()
    plt.show()
    
    print(f"Group: {group_name}, Optimal k (gap): {optimal_num_k[group_name]}, Optimal k (gap*): {optimal_num_k_2[group_name]}")
    '''


 21%|██▏       | 689/3233 [50:43<2:46:33,  3.93s/it]/var/folders/41/xvr_rn1j51vd_js7xks_mznh0000gn/T/ipykernel_76671/1129347772.py:18: RuntimeWarning: divide by zero encountered in log
  gap = np.mean(np.log(wcss_null)) - np.log(wcss)
/var/folders/41/xvr_rn1j51vd_js7xks_mznh0000gn/T/ipykernel_76671/1129347772.py:18: RuntimeWarning: invalid value encountered in double_scalars
  gap = np.mean(np.log(wcss_null)) - np.log(wcss)
/var/folders/41/xvr_rn1j51vd_js7xks_mznh0000gn/T/ipykernel_76671/1129347772.py:21: RuntimeWarning: divide by zero encountered in log
  gap_std = np.std(np.log(wcss_null))
/Users/julienmbarki/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
 21%|██▏       | 690/3233 [50:46<2:33:03,  3.61s/it]/var/folders/41/xvr_rn1j51vd_js7xks_mznh0000gn/T/ipykernel_76671/1129347772.py:18: RuntimeWarning: divide by zero encountered in log
  gap = np.mean(np.log(wcss_null)) - np.

### Apply KMeans

In [11]:
# Apply KMeans with optimal number of clusters
df = pd.DataFrame()

kmeans_dict = {}
kmeans_2_dict = {}
cluster_labels_dict = {}
cluster_labels_2_dict = {}

for group_name, group in groups:
    reduced_data = reduced_data_dict[group_name]

    k = optimal_num_k[group_name]
    k_2 = optimal_num_k_2[group_name]

    # Apply KMeans with optimal number of clusters
    kmeans = KMeans(n_clusters=k, random_state=42).fit(reduced_data)
    kmeans_2 = KMeans(n_clusters=k_2, random_state=42).fit(reduced_data)
        
    kmeans_dict[group_name] = kmeans
    kmeans_2_dict[group_name] = kmeans_2

    cluster_labels_dict[group_name] = kmeans.labels_
    cluster_labels_2_dict[group_name] = kmeans_2.labels_

    print(f"Group {group_name}: {kmeans.labels_}")

    # Assign cluster labels to original group data
    group = group.assign(cluster=kmeans.labels_)
    group = group.assign(cluster_2=kmeans_2.labels_)

    # Append group to original dataframe
    df = pd.concat([df, group])
    
    # Plot clusters
    '''
    plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=kmeans.labels_, cmap='viridis')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title(group_name)
    plt.show()
    '''
display(df)


Group ('#NUITVIE', '2021-10-01 00:00:00+00:00'): [1 0 0 1 1 0 1 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 1 1 0 0
 0 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 0
 1 1 1 1 1 0 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 0 0 1 1 0 1 0 1 1 0 0 0 0 1
 0 0 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 1 0 0 0 0 1 1
 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0]
Group ('#NUITVIE', '2021-10-08 00:00:00+00:00'): [3 0 1 3 3 1 3 0 0 0 0 1 1 2 3 1 1 2 0 1 1 2 0 3 0 0 0 1 2 2 2 0 0 3 0 1 0
 0 3 0 0 0 0 3 0 0 1 0 3 0 3 1 0 1 0 0 0 0 1 0 0 3 1 3 3 3 3 0 3 0 3 0 1 1
 2 1 1 1 2 1 0 1 1 1 1 2 2 1 1 1 2 2 1 1 3 1 1 1 3 1 0 3 0 2 2 1 3 2 3 0 3
 1 3 0 0 0 2 1 0 0 2 1 0 3 1 2 1 0 3 0 1 3 3 1 2 1 0 0 0 1 3 0 0 3 0 1 1 1
 0 1 1 1 0 2 1 2 0 1 2 2 1 1 1 3 0 

,playlist_id,playlist_name,collection_date,playlist_type,track_position,track_name,track_uuid,first_appearance_date,artist_name,artist_id,...,key,liveness,loudness,mode,speechiness,tempo,timeSignature,valence,cluster,cluster_2
272611,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-01 00:00:00+00:00,Major label,1,L'amour du vide,4e28b8c2-cf17-11e9-abbf-549f35141000,2021-10-01 00:00:00+00:00,['HATIK'],['11e81bd2-48d9-6ce4-b69e-a0369fe50396'],...,3.0,0.16,-7.21,0.0,0.05,127.97,4.0,0.50,1,5
272612,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-01 00:00:00+00:00,Major label,2,XNXX,00a05a0e-e62a-11e9-a9d6-e4434b16cb78,2021-10-01 00:00:00+00:00,['Dinos'],['11e83fed-a370-a0ac-9db2-a0369fe50396'],...,2.0,0.22,-9.47,0.0,0.29,132.04,4.0,0.36,0,0
272613,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-01 00:00:00+00:00,Major label,3,Paire d'As,5f8644b0-0769-11ea-a108-801844ee83a6,2021-10-01 00:00:00+00:00,"['Dadju', 'Nekfeu']","['11e81bc8-cb77-940a-9131-a0369fe50396', '11e8...",...,1.0,0.09,-6.98,0.0,0.14,86.03,4.0,0.31,0,2
272614,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-01 00:00:00+00:00,Major label,4,Machine à sous,2d8c998e-d4a9-11e9-83fd-549f35141000,2021-10-01 00:00:00+00:00,['HATIK'],['11e81bd2-48d9-6ce4-b69e-a0369fe50396'],...,10.0,0.10,-9.72,0.0,0.23,102.91,4.0,0.23,1,8
272616,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-01 00:00:00+00:00,Major label,6,Belle en noir,1296c92a-caf3-11e9-a204-e4434b16cb7a,2021-10-01 00:00:00+00:00,['HATIK'],['11e81bd2-48d9-6ce4-b69e-a0369fe50396'],...,11.0,0.10,-9.78,0.0,0.05,80.05,4.0,0.19,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374662,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,106,Acapulco,bf677c95-c9eb-4e09-a815-b42c80a705f6,2021-10-01 00:00:00+00:00,['Jason Derulo'],['11e81bcc-e6ef-4616-a1c3-a0369fe50396'],...,10.0,0.16,-4.02,1.0,0.05,122.06,4.0,0.51,0,5
374663,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,107,Mon Amour - Remix,29a0db26-4828-45dd-9299-4ea8d659c107,2021-12-04 00:00:00+00:00,"['Aitana', 'zzoilo']","['11e83fea-f894-b5e4-9144-a0369fe50396', '8702...",...,2.0,0.10,-6.62,1.0,0.10,116.04,4.0,0.36,1,1
374664,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,108,Sans moi,be9cc621-a5fb-4096-82cd-912367f155c5,2021-10-01 00:00:00+00:00,"['Franglish', 'Aya Nakamura']","['11e81bbb-8b2f-cc8a-98a2-a0369fe50396', '11e8...",...,1.0,0.09,-4.45,1.0,0.33,91.79,4.0,0.42,1,0
374665,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-30 00:00:00+00:00,Major label,109,Better Days,c1f601c2-785f-41cd-8171-f8d421fd27a8,2022-01-30 00:00:00+00:00,"['J Balvin', 'NEIKED', 'Mae Muller', 'Polo G']","['11e81bca-6acb-9a46-8802-a0369fe50396', '11e8...",...,0.0,0.10,-5.09,0.0,0.04,110.03,4.0,0.75,0,5


## Variables creation

### HH-Index

In [12]:
groups = df.groupby(['playlist_name', 'collection_date'], as_index=False)


In [13]:
# Calculate the shares of each cluster
shares_perc = {}

for group_name, group in groups:
    shares = group['cluster'].value_counts(normalize=True)
    shares_perc[group_name] = shares * 100

# Calculate the HHI
hhi = {}
hhi_2 = {}

for group_name, group in groups:
  shares = group['cluster'].value_counts(normalize=True)
  hhi[group_name] = sum((shares*100)**2)

  shares_2 = group['cluster_2'].value_counts(normalize=True)
  hhi_2[group_name] = sum((shares_2*100)**2)

print(hhi)
print(hhi_2)

print("Minimum HHI: ", min(hhi.values()))
print("Mean HHI: ", np.mean(list(hhi.values())))
print("Median HHI: ", np.median(list(hhi.values())))
print("Maximum HHI: ", max(hhi.values()))
print("Standard deviation: ", np.std(list(hhi.values())))


{('#NUITVIE', '2021-10-01 00:00:00+00:00'): 5033.979522359682, ('#NUITVIE', '2021-10-08 00:00:00+00:00'): 2798.3967317994375, ('#NUITVIE', '2021-10-15 00:00:00+00:00'): 3419.920093523978, ('#NUITVIE', '2021-10-22 00:00:00+00:00'): 2798.3967317994375, ('#NUITVIE', '2021-10-29 00:00:00+00:00'): 2798.3967317994375, ('#NUITVIE', '2021-11-05 00:00:00+00:00'): 3419.920093523978, ('#NUITVIE', '2021-11-12 00:00:00+00:00'): 2798.3967317994375, ('#NUITVIE', '2021-11-19 00:00:00+00:00'): 2798.3967317994375, ('#NUITVIE', '2021-11-26 00:00:00+00:00'): 3419.920093523978, ('#NUITVIE', '2021-12-03 00:00:00+00:00'): 3419.920093523978, ('#NUITVIE', '2021-12-10 00:00:00+00:00'): 2798.3967317994375, ('#NUITVIE', '2021-12-17 00:00:00+00:00'): 2798.3967317994375, ('#NUITVIE', '2021-12-24 00:00:00+00:00'): 3419.920093523978, ('#NUITVIE', '2021-12-31 00:00:00+00:00'): 3419.920093523978, ('#NUITVIE', '2022-01-07 00:00:00+00:00'): 3419.920093523978, ('#NUITVIE', '2022-01-14 00:00:00+00:00'): 3419.920093523978, 

### Euclidean distances

In [14]:
# Calculate distances between cluster centroids
distances_dict = {}
distances_dict_2 = {}

for group_name, group in groups:
    # Get the cluster centroids for the group
    cluster_centroids = kmeans_dict[group_name].cluster_centers_
    cluster_centroids_2 = kmeans_2_dict[group_name].cluster_centers_

    # Calculate pairwise euclidean distances between cluster centroids
    distances = pairwise_distances(cluster_centroids)
    distances_2 = pairwise_distances(cluster_centroids_2)

    # Calculate mean pairwise distance
    mean_distance = distances.mean()
    mean_distance_2 = distances_2.mean()

    # Print mean pairwise distance for the group
    print(f"Group {group_name}: Mean pairwise distance between cluster centroids = {mean_distance}")

    distances_dict[group_name] = mean_distance
    distances_dict_2[group_name] = mean_distance_2

# Calculate max, min, mean, median and standard deviation of distances
distances_list = [d for distances in distances_dict.values() for d in distances.flatten() if not np.isnan(d)]
print(f"Max distance: {np.max(distances_list):.5f}")
print(f"Min distance: {np.min(distances_list):.5f}")
print(f"Mean distance: {np.mean(distances_list):.5f}")
print(f"Median distance: {np.median(distances_list):.5f}")
print(f"Standard deviation of distances: {np.std(distances_list):.5f}")


Group ('#NUITVIE', '2021-10-01 00:00:00+00:00'): Mean pairwise distance between cluster centroids = 0.23750550893009378
Group ('#NUITVIE', '2021-10-08 00:00:00+00:00'): Mean pairwise distance between cluster centroids = 0.4146617735687224
Group ('#NUITVIE', '2021-10-15 00:00:00+00:00'): Mean pairwise distance between cluster centroids = 0.353582657122123
Group ('#NUITVIE', '2021-10-22 00:00:00+00:00'): Mean pairwise distance between cluster centroids = 0.4146617735687224
Group ('#NUITVIE', '2021-10-29 00:00:00+00:00'): Mean pairwise distance between cluster centroids = 0.4146617735687224
Group ('#NUITVIE', '2021-11-05 00:00:00+00:00'): Mean pairwise distance between cluster centroids = 0.353582657122123
Group ('#NUITVIE', '2021-11-12 00:00:00+00:00'): Mean pairwise distance between cluster centroids = 0.4146617735687224
Group ('#NUITVIE', '2021-11-19 00:00:00+00:00'): Mean pairwise distance between cluster centroids = 0.4146617735687224
Group ('#NUITVIE', '2021-11-26 00:00:00+00:00'): 

In [15]:
# Calculate distances bewteen tracks
scaler = MinMaxScaler()

selected_columns = [
    "danceability", "energy", "loudness", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", 
    "tempo"
]

distances_dict_3 = {}

for group_name, group in groups:
    subset = group.loc[:, selected_columns]
    scaled_columns = scaler.fit_transform(subset)

    pairwise_dist = pairwise_distances(scaled_columns)
    mean_distance = pairwise_dist.mean()
    
    # Print mean pairwise distance for the group
    print(f"Group {group_name}: Mean pairwise distance between tracks = {mean_distance}")

    distances_dict_3[group_name] = mean_distance

# Calculate max, min, mean, median and standard deviation of distances
distances_list = [d for distances in distances_dict_3.values() for d in distances.flatten() if not np.isnan(d)]
print(f"Max distance: {np.max(distances_list):.5f}")
print(f"Min distance: {np.min(distances_list):.5f}")
print(f"Mean distance: {np.mean(distances_list):.5f}")
print(f"Median distance: {np.median(distances_list):.5f}")
print(f"Standard deviation of distances: {np.std(distances_list):.5f}")


Group ('#NUITVIE', '2021-10-01 00:00:00+00:00'): Mean pairwise distance between tracks = 0.7687364732143525
Group ('#NUITVIE', '2021-10-08 00:00:00+00:00'): Mean pairwise distance between tracks = 0.7690208837053262
Group ('#NUITVIE', '2021-10-15 00:00:00+00:00'): Mean pairwise distance between tracks = 0.7690208837053262
Group ('#NUITVIE', '2021-10-22 00:00:00+00:00'): Mean pairwise distance between tracks = 0.7690208837053262
Group ('#NUITVIE', '2021-10-29 00:00:00+00:00'): Mean pairwise distance between tracks = 0.7690208837053262
Group ('#NUITVIE', '2021-11-05 00:00:00+00:00'): Mean pairwise distance between tracks = 0.7690208837053262
Group ('#NUITVIE', '2021-11-12 00:00:00+00:00'): Mean pairwise distance between tracks = 0.7690208837053262
Group ('#NUITVIE', '2021-11-19 00:00:00+00:00'): Mean pairwise distance between tracks = 0.7690208837053262
Group ('#NUITVIE', '2021-11-26 00:00:00+00:00'): Mean pairwise distance between tracks = 0.7690208837053262
Group ('#NUITVIE', '2021-12-

### Stirling diversity index

In [16]:
# k-alpha Rao-Stirling index
stirling_index_dict = {}

for group_name, group in groups:
    clusters = group.groupby('cluster')

    total_index = 0

    cluster_centroids = kmeans_dict[group_name].cluster_centers_
    
    for i, (cluster_i_name, cluster_i) in enumerate(clusters):
        for j, (cluster_j_name, cluster_j) in enumerate(clusters):
            if i >= j:
                continue
            
            centroid_i = cluster_centroids[i]
            centroid_j = cluster_centroids[j]
            dist = cdist([centroid_i], [centroid_j], 'euclidean')

            share_i = len(cluster_i) / len(group)
            share_j = len(cluster_j) / len(group)

            index = dist * share_i * share_j

            total_index += index

    stirling_index_dict[group_name] = total_index

print(stirling_index_dict)

print("Minimum index: ", min(stirling_index_dict.values()))
print("Mean index: ", np.mean(list(stirling_index_dict.values())))
print("Median index: ", np.median(list(stirling_index_dict.values())))
print("Maximum index: ", max(stirling_index_dict.values()))
print("Standard deviation: ", np.std(list(stirling_index_dict.values())))


{('#NUITVIE', '2021-10-01 00:00:00+00:00'): array([[0.11794572]]), ('#NUITVIE', '2021-10-08 00:00:00+00:00'): array([[0.19764525]]), ('#NUITVIE', '2021-10-15 00:00:00+00:00'): array([[0.17317509]]), ('#NUITVIE', '2021-10-22 00:00:00+00:00'): array([[0.19764525]]), ('#NUITVIE', '2021-10-29 00:00:00+00:00'): array([[0.19764525]]), ('#NUITVIE', '2021-11-05 00:00:00+00:00'): array([[0.17317509]]), ('#NUITVIE', '2021-11-12 00:00:00+00:00'): array([[0.19764525]]), ('#NUITVIE', '2021-11-19 00:00:00+00:00'): array([[0.19764525]]), ('#NUITVIE', '2021-11-26 00:00:00+00:00'): array([[0.17317509]]), ('#NUITVIE', '2021-12-03 00:00:00+00:00'): array([[0.17317509]]), ('#NUITVIE', '2021-12-10 00:00:00+00:00'): array([[0.19764525]]), ('#NUITVIE', '2021-12-17 00:00:00+00:00'): array([[0.19764525]]), ('#NUITVIE', '2021-12-24 00:00:00+00:00'): array([[0.17317509]]), ('#NUITVIE', '2021-12-31 00:00:00+00:00'): array([[0.17317509]]), ('#NUITVIE', '2022-01-07 00:00:00+00:00'): array([[0.17317509]]), ('#NUITVI

In [17]:
# k-beta Rao-Stirling index
stirling_index_2_dict = {}

for group_name, group in groups:
    clusters = group.groupby('cluster_2')

    total_index = 0

    cluster_centroids = kmeans_2_dict[group_name].cluster_centers_
    
    for i, (cluster_i_name, cluster_i) in enumerate(clusters):
        for j, (cluster_j_name, cluster_j) in enumerate(clusters):
            if i >= j:
                continue
            
            centroid_i = cluster_centroids[i]
            centroid_j = cluster_centroids[j]
            dist = cdist([centroid_i], [centroid_j], 'euclidean')

            share_i = len(cluster_i) / len(group)
            share_j = len(cluster_j) / len(group)

            index = dist * share_i * share_j

            total_index += index

    stirling_index_2_dict[group_name] = total_index

print(stirling_index_2_dict)

print("Minimum index: ", min(stirling_index_2_dict.values()))
print("Mean index: ", np.mean(list(stirling_index_2_dict.values())))
print("Median index: ", np.median(list(stirling_index_2_dict.values())))
print("Maximum index: ", max(stirling_index_2_dict.values()))
print("Standard deviation: ", np.std(list(stirling_index_2_dict.values())))


{('#NUITVIE', '2021-10-01 00:00:00+00:00'): array([[0.25934183]]), ('#NUITVIE', '2021-10-08 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-10-15 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-10-22 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-10-29 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-11-05 00:00:00+00:00'): array([[0.25448635]]), ('#NUITVIE', '2021-11-12 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-11-19 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-11-26 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-12-03 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-12-10 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-12-17 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-12-24 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2021-12-31 00:00:00+00:00'): array([[0.26065338]]), ('#NUITVIE', '2022-01-07 00:00:00+00:00'): array([[0.24134023]]), ('#NUITVI

## Append to dataframe

In [18]:
# Append playlist-level indicators
data_final = []

for group_name, group in groups:
    # Variety
    nb_clusters = optimal_num_k[group_name]
    nb_clusters_2 = optimal_num_k_2[group_name]

    # Balance
    hh_index = hhi[group_name]
    hh_index_2 = hhi_2[group_name]

    # Distparity
    distances = distances_dict[group_name]
    distances_2 = distances_dict_2[group_name]
    distances_3 = distances_dict_3[group_name]

    # Diversity
    stirling_index = stirling_index_dict[group_name]
    stirling_index_2 = stirling_index_2_dict[group_name]
    
    # Metadata playlist
    playlist_id = group['playlist_id'].iloc[0]
    playlist_name = group['playlist_name'].iloc[0]
    playlist_type = group['playlist_type'].iloc[0]
    collection_date = group['collection_date'].iloc[0]
    nb_tracks = len(group)

    dates = pd.to_datetime(group['track_date'])
    mean_track_date = dates.mean()

    # Append playlist-level indicators
    data_final.append({
        'playlist_id': playlist_id,
        'playlist_name': playlist_name,
        'collection_date': collection_date,
        'mean_track_date': mean_track_date,
        'playlist_type': playlist_type,
        'nb_tracks': nb_tracks,
        'nb_clusters': nb_clusters,
        'nb_clusters_2': nb_clusters_2,
        'hh_index': hh_index,
        'hh_index_2': hh_index_2,
        'distances': distances,
        'distances_2': distances_2,
        'distances_3': distances_3,
        'stirling_index': stirling_index,
        'stirling_index_2': stirling_index_2
    })

# Convert to DataFrame
final_data_df = pd.DataFrame(data_final)
display(final_data_df)


,playlist_id,playlist_name,collection_date,mean_track_date,playlist_type,nb_tracks,nb_clusters,nb_clusters_2,hh_index,hh_index_2,distances,distances_2,distances_3,stirling_index,stirling_index_2
0,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-01 00:00:00+00:00,2017-06-22 06:32:15.483870976+00:00,Major label,279,2,10,5033.979522,1158.386968,0.237506,0.568287,0.768736,[[0.11794572208992313]],[[0.25934182502566594]]
1,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-08 00:00:00+00:00,2017-06-11 17:38:03.870967808+00:00,Major label,279,4,10,2798.396732,1080.022096,0.414662,0.548911,0.769021,[[0.1976452498502834]],[[0.26065337521371623]]
2,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-15 00:00:00+00:00,2017-06-11 17:38:03.870967808+00:00,Major label,279,3,10,3419.920094,1080.022096,0.353583,0.548911,0.769021,[[0.17317508694841288]],[[0.26065337521371623]]
3,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-22 00:00:00+00:00,2017-06-11 17:38:03.870967808+00:00,Major label,279,4,10,2798.396732,1080.022096,0.414662,0.548911,0.769021,[[0.1976452498502834]],[[0.26065337521371623]]
4,11e84480-ad5c-55c4-a92d-a0369fe50396,#NUITVIE,2021-10-29 00:00:00+00:00,2017-06-11 17:38:03.870967808+00:00,Major label,279,4,10,2798.396732,1080.022096,0.414662,0.548911,0.769021,[[0.1976452498502834]],[[0.26065337521371623]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-02 00:00:00+00:00,2021-01-14 19:09:21.467889920+00:00,Major label,109,2,8,5121.622759,1850.854305,0.238898,0.656555,0.781643,[[0.11654333088871671]],[[0.2575646520628176]]
3229,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-09 00:00:00+00:00,2021-01-14 19:09:21.467889920+00:00,Major label,109,2,8,5121.622759,1850.854305,0.238898,0.656555,0.781643,[[0.11654333088871671]],[[0.2575646520628176]]
3230,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-16 00:00:00+00:00,2021-01-14 19:09:21.467889920+00:00,Major label,109,2,8,5121.622759,1850.854305,0.238898,0.656555,0.781643,[[0.11654333088871671]],[[0.2575646520628176]]
3231,11e84480-ad6e-2cf4-a2cd-a0369fe50396,"Été 2024 ☀️ Piscine, soleil & fête | Summer hi...",2022-09-23 00:00:00+00:00,2021-01-14 19:09:21.467889920+00:00,Major label,109,2,8,5121.622759,1850.854305,0.238898,0.656555,0.781643,[[0.11654333088871671]],[[0.2575646520628176]]


In [19]:
# Export data
final_data_df.to_csv("major_playlists_21-22_final.csv", index=False)
